The following cell establishes a connection to Mechanical Turk.   
Set the create_hits_in_production variable equal to true to public real tasks.  
Remember to change the AWSCLI credentials to fit whatever account you use. 

In [59]:
# CREATING CLIENT CONNECTION TO MTURK 

import boto3 
import xmltodict 
import json 

create_hits_in_production = False
environments = {
  "production": {
    "endpoint": "https://mturk-requester.us-east-1.amazonaws.com",
    "preview": "https://www.mturk.com/mturk/preview"
  },
  "sandbox": {
    "endpoint": 
          "https://mturk-requester-sandbox.us-east-1.amazonaws.com",
    "preview": "https://workersandbox.mturk.com/mturk/preview"
  },
}
mturk_environment = environments["production"] if create_hits_in_production else environments["sandbox"]
session = boto3.Session(profile_name='mturk')
client = session.client(
    service_name='mturk',
    region_name='us-east-1',
    endpoint_url=mturk_environment['endpoint'],
)

The following cell checks the account balance. 

In [60]:
# CHECKING ACCOUNT BALANCE 

print(client.get_account_balance()['AvailableBalance'])

10000.00


The following cell outlines the global variables for task creation. 

In [61]:
# GLOBAL TASK ATTRIBUTES 

max_assignments = 40 # we want a total of 40 assigments where each assignment contains 5 images to evaluate 
lifetime = 60*60*24 # the HITs will be up for a total of 24 hours (this value is in seconds)
duration = 60*60
reward = '0.25'
title = 'Provide Your Rating for a Work of Art'
keywords = 'artwork, rating'
description = 'Rate the artwork on a scale of 1 to 10'

The following cell contains the information about the images for the tasks. 

In [62]:
images = {
    "https://i.imgur.com/POfoxMI.jpeg":"Kalina",
    "https://preview.redd.it/kzqzeaefm1c51.jpg?width=640&crop=smart&auto=webp&s=e030a71738158154534ed15c8fcdc338bffdc0d1":"Richers",
    "https://i.imgur.com/quUHmVg.png":"Whiteford",
    "https://miro.medium.com/max/700/1*N8OLmyNYXgk-eGrc6St7rA.jpeg":"Delaroche",
    "https://upload.wikimedia.org/wikipedia/commons/0/07/CAPPELLA_SISTINA_Ceiling.jpg":"Michelangelo"
}

descriptions = {
    "https://i.imgur.com/POfoxMI.jpeg":"This work of art was painted by Kalina Harden in 2020.  Harden is a 22 year old psychology and economics major at Williams College who enjoys painting in her free time.",
    "https://preview.redd.it/kzqzeaefm1c51.jpg?width=640&crop=smart&auto=webp&s=e030a71738158154534ed15c8fcdc338bffdc0d1":"This work of art was painted by Massimo Richers in 2020.  Richers is a 21 year old professional painter from Victoria, British Columbia.",
    "https://i.imgur.com/quUHmVg.png":"This work of art was painted by Blair Whiteford in 2021.  Whiteford is a professional painter who graduated from Yale School of Art in 2019.  This work now hangs at the Fountainhead Residency in Miami, Florida.",
    "https://miro.medium.com/max/700/1*N8OLmyNYXgk-eGrc6St7rA.jpeg":"This work of art was painted by Paul Delaroche in 1855.  Delaroche was a leading pupil of fellow French painter Antoine-Jean Gros.  This work now hangs in the Louvre, Paris.",
    "https://upload.wikimedia.org/wikipedia/commons/0/07/CAPPELLA_SISTINA_Ceiling.jpg":"This work of art was painted by Michelangelo around 1510.  Michelangelo was an Italian painter during the High Renaissance in Italy.  This work is on the ceiling of the Sistine Chapel in Vatican City."
}

The follwing cell creates HITs for the non-expert task version. 

In [63]:
html_layout = open('./arteval.html', 'r').read()

QUESTION_XML = """<HTMLQuestion xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2011-11-11/HTMLQuestion.xsd">
        <HTMLContent><![CDATA[{}]]></HTMLContent>
        <FrameHeight>650</FrameHeight>
        </HTMLQuestion>"""

question_xml = QUESTION_XML.format(html_layout)

TaskAttributes = {
    'MaxAssignments': max_assignments,           
    # How long the task will be available on MTurk     
    'LifetimeInSeconds': lifetime,
    # How long Workers have to complete each item 
    'AssignmentDurationInSeconds': duration,
    # The reward you will offer Workers for each response
    'Reward': reward,                     
    'Title': title+'(Non-Expert)',
    'Keywords': keywords,
    'Description': description
}

results = []
hit_type_id = ''
for image in images:
    response = client.create_hit(
        **TaskAttributes,
        Question=question_xml.replace('${content}',image).replace('${description}','')
    )
    hit_type_id = response['HIT']['HITTypeId']
    results.append({
        'image': images[image],
        'hit_id': response['HIT']['HITId']
    })
    
print("You can view the HITs here:")
print(mturk_environment['preview']+"?groupId={}".format(hit_type_id))

You can view the HITs here:
https://workersandbox.mturk.com/mturk/preview?groupId=3D9187VUCT9B746PMO80E4H9BXYUIY


The following cell prints the results of the non-expert HIT. The code is identical to retrieving the results of the expert task.

In [64]:
for item in results:
    
    # Get the status of the HIT
    hit = client.get_hit(HITId=item['hit_id'])
    item['status'] = hit['HIT']['HITStatus']
    # Get a list of the Assignments that have been submitted
    assignmentsList = client.list_assignments_for_hit(
        HITId=item['hit_id'],
        AssignmentStatuses=['Submitted', 'Approved'],
        MaxResults=10
    )
    assignments = assignmentsList['Assignments']
    item['assignments_submitted_count'] = len(assignments)
    answers = []
    responses = [] 
    for assignment in assignments:
        # Retreive the attributes for each Assignment
        worker_id = assignment['WorkerId']
        assignment_id = assignment['AssignmentId']
        # Retrieve the value submitted by the Worker from the XML
        answer_dict = xmltodict.parse(assignment['Answer'])
        # print(answer_dict['QuestionFormAnswers']['Answer'][0]['FreeText']) # text 
        # print(answer_dict['QuestionFormAnswers']['Answer'][1]['FreeText']) # rating 
        response = answer_dict['QuestionFormAnswers']['Answer'][0]['FreeText']
        responses.append(response)
        answer = answer_dict['QuestionFormAnswers']['Answer'][1]['FreeText']
        answers.append(int(answer))
        
        # Approve the Assignment (if it hasn't been already)
        if assignment['AssignmentStatus'] == 'Submitted':
            client.approve_assignment(
                AssignmentId=assignment_id,
                OverrideRejection=False
            )
    
    # Add the answers that have been retrieved for this item
    item['answers'] = answers
    item['responses'] = responses 
    if len(answers) > 0:
        item['avg_answer'] = sum(answers)/len(answers)
print(json.dumps(results,indent=2))

[
  {
    "image": "Kalina",
    "hit_id": "3CZH926SJJ4HKA16H7MBV5SX4TK4EI",
    "status": "Assignable",
    "assignments_submitted_count": 0,
    "answers": [],
    "responses": []
  },
  {
    "image": "Richers",
    "hit_id": "3SZYX62S6NQE7EQW84RFUN6UI2B754",
    "status": "Assignable",
    "assignments_submitted_count": 0,
    "answers": [],
    "responses": []
  },
  {
    "image": "Whiteford",
    "hit_id": "3QGHA0EA1QQVM986HMQ4PP53LP6BWB",
    "status": "Assignable",
    "assignments_submitted_count": 0,
    "answers": [],
    "responses": []
  },
  {
    "image": "Delaroche",
    "hit_id": "3SBNLSTU71VSV1BV651HJJNL5CMDZA",
    "status": "Assignable",
    "assignments_submitted_count": 0,
    "answers": [],
    "responses": []
  },
  {
    "image": "Michelangelo",
    "hit_id": "3ZRKL6Z1FFTY69Z8D38HTBTFVP2GSI",
    "status": "Assignable",
    "assignments_submitted_count": 0,
    "answers": [],
    "responses": []
  }
]


The below cell implements and creates HITs for the expert task. 

In [65]:
html_layout = open('./arteval.html', 'r').read()
QUESTION_XML = """<HTMLQuestion xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2011-11-11/HTMLQuestion.xsd">
        <HTMLContent><![CDATA[{}]]></HTMLContent>
        <FrameHeight>650</FrameHeight>
        </HTMLQuestion>"""
question_xml = QUESTION_XML.format(html_layout)

TaskAttributes = {
    'MaxAssignments': max_assignments,           
    # How long the task will be available on MTurk     
    'LifetimeInSeconds': lifetime,
    # How long Workers have to complete each item 
    'AssignmentDurationInSeconds': duration,
    # The reward you will offer Workers for each response
    'Reward': reward,                     
    'Title': title+'(Expert)',
    'Keywords': keywords,
    'Description': description
}

results = []
hit_type_id = ''
for image in images:
    response = client.create_hit(
        **TaskAttributes,
        Question=question_xml.replace('${content}',image).replace('${description}',descriptions[image])
    )
    hit_type_id = response['HIT']['HITTypeId']
    results.append({
        'image': images[image],
        'hit_id': response['HIT']['HITId']
    })
    
print("You can view the HITs here:")
print(mturk_environment['preview']+"?groupId={}".format(hit_type_id))

You can view the HITs here:
https://workersandbox.mturk.com/mturk/preview?groupId=3CU3NNU95EA4JP31VS1ZZI0QBLULJ7


In [66]:
for item in results:
    
    # Get the status of the HIT
    hit = client.get_hit(HITId=item['hit_id'])
    item['status'] = hit['HIT']['HITStatus']
    # Get a list of the Assignments that have been submitted
    assignmentsList = client.list_assignments_for_hit(
        HITId=item['hit_id'],
        AssignmentStatuses=['Submitted', 'Approved'],
        MaxResults=10
    )
    assignments = assignmentsList['Assignments']
    item['assignments_submitted_count'] = len(assignments)
    answers = []
    responses = [] 
    for assignment in assignments:
        # Retreive the attributes for each Assignment
        worker_id = assignment['WorkerId']
        assignment_id = assignment['AssignmentId']
        # Retrieve the value submitted by the Worker from the XML
        answer_dict = xmltodict.parse(assignment['Answer'])
        # print(answer_dict['QuestionFormAnswers']['Answer'][0]['FreeText']) # text 
        # print(answer_dict['QuestionFormAnswers']['Answer'][1]['FreeText']) # rating 
        response = answer_dict['QuestionFormAnswers']['Answer'][0]['FreeText']
        responses.append(response)
        answer = answer_dict['QuestionFormAnswers']['Answer'][1]['FreeText']
        answers.append(int(answer))
        
        # Approve the Assignment (if it hasn't been already)
        if assignment['AssignmentStatus'] == 'Submitted':
            client.approve_assignment(
                AssignmentId=assignment_id,
                OverrideRejection=False
            )
    
    # Add the answers that have been retrieved for this item
    item['answers'] = answers
    item['responses'] = responses 
    if len(answers) > 0:
        item['avg_answer'] = sum(answers)/len(answers)
print(json.dumps(results,indent=2))

[
  {
    "image": "Kalina",
    "hit_id": "36818Z1KWA3TC11Q1OF6PHA05EQA38",
    "status": "Assignable",
    "assignments_submitted_count": 0,
    "answers": [],
    "responses": []
  },
  {
    "image": "Richers",
    "hit_id": "3VMV5CHJ0F5ZL9CN23D73GP1X8GGT7",
    "status": "Assignable",
    "assignments_submitted_count": 0,
    "answers": [],
    "responses": []
  },
  {
    "image": "Whiteford",
    "hit_id": "3UV0D2KX2T99QO12CI6HDNDXGUZ4F0",
    "status": "Assignable",
    "assignments_submitted_count": 0,
    "answers": [],
    "responses": []
  },
  {
    "image": "Delaroche",
    "hit_id": "3KI0JD2ZV88AQIQADN1KMTKUEXM76K",
    "status": "Assignable",
    "assignments_submitted_count": 0,
    "answers": [],
    "responses": []
  },
  {
    "image": "Michelangelo",
    "hit_id": "3ICOHX7EOJ10LMJ5Y6Z9J22KNH4E0C",
    "status": "Assignable",
    "assignments_submitted_count": 0,
    "answers": [],
    "responses": []
  }
]
